In [14]:
#set input filepath in relation to this notebook
asdr_filename = 'DVO/ASDR.csv'

In [ ]:
#set whether to print data for debugging
DEBUG = False

In [15]:
# This code block installs all the necessary libraries
# Only needs to be run once, set to False after initial run
IF_INITAL_RUN = False
if IF_INITAL_RUN:
    %pip install pandas
    %pip install dict2xml

In [16]:
# import libraries
import pandas as pd
from dict2xml import dict2xml
from xml.dom import minidom
import xml.etree.ElementTree as ET

In [17]:
# Count the unique columns and save their names
with open(asdr_filename, 'r') as temp_f:
    col_count = [ len(l.split(",")) for l in temp_f.readlines() ]
column_names = [i for i in range(0, max(col_count))]

In [ ]:
# Read csv
asdr_df = pd.read_csv(asdr_filename, header=None, delimiter=",", names=column_names)

In [ ]:
# input CSV to excel
generate_excel = False
if generate_excel:
    asdr_df.to_excel(str((asdr_filename.split('.')[0] + ".xlsx")),header=False,index=False)

In [ ]:
# create systems as nested dictionaries
# this can take some time to run
systems = []
for row in range(len(asdr_df.index)):
    if asdr_df.iloc[row][0] == 'TEC System Name:' or asdr_df.iloc[row][0] == 'Zone System Name:':
        if row > 3:
            systems.append(system_info)
        system_info = {}
        system_info['NAME'] = asdr_df.iloc[row][1].replace('.','_')
    elif not pd.isna(asdr_df.iloc[row][1]):        
        system_info[str(asdr_df.iloc[row][1]).split(":")[1]] = dict(zip(['VALUE','UNIT','STATUS'], asdr_df.iloc[row][4:7]))

systems.append(system_info)       
systems

In [ ]:
# find unique entries
output_df = pd.read_xml("DVO\Bacnet-Export.xml")
output_df
for column in output_df.columns:
    if len(output_df[column].unique()) < 100:
        print(column,output_df[column].unique())
    else:
        print(column)

In [ ]:
# make Names
mappings =  ['ROOM_TEMP', 'DMPR_COMD', 'AIR_VOLUME', 'CTL_STPT', 'AI_3',
             'VLV_COMD', 'CTL_TEMP_IGNORE', 'SUPPLY_TEMP', 'MTR2_COMD', 'DMPR_POS_IGNORE',
             'AUX_TEMP', 'VLV1_COMD']
mapping_codes = ['004','048','035','092','015','052','078','015','052','049','015','052']

mapping_spaces = []
combined_mappings = []

for i in range(len(mappings)):
    mapping_spaces.append(mappings[i].replace("_"," "))
    combined_mappings.append('(' + mapping_codes[i] + ') ' + mapping_spaces[i])
combined_mappings


In [21]:
# build default system info
output_columns = ['Inst', 'Name', 'Desc', 'NC', 'Type', 'Enabled', 'Alarm', 'OM', 'MappedDp', 'MappedDpe','ZeroBase']
default_values = [None,None,None,'1',"Analog-Value",'true','true',None,None,None,'false']
inst_counter = 1

dev_attrib = dict(zip(["Inst","Name","Desc","Enabled","ReadOnly"],
                      ["1","VirtualBACnetDevice","Virtual BACnet Device","true","true"]))


output_df = pd.DataFrame(columns=output_columns)
root = ET.Element('DEV',dev_attrib)

In [22]:
# build element tree
for system in systems:
    for key in list(system.keys()):
        
        if key in mapping_spaces:
            current_row = default_values

            # Inst
            current_row[0] = str(inst_counter)
            inst_counter += 1

            # Name
            current_row[1] = str(system['NAME']) + " - " + combined_mappings[mapping_spaces.index(key)]

            # Desc
            current_row[2] = system['NAME']
            
            # OM
            current_row[7] = ("GMS_APOGEE_TEC_APPL_" + str(int(float(system['APPLICATION']['VALUE']))) + "_EN")
            
            # MappedDp
            current_row[8] = ("GMS_AP2_" + ((system['NAME']).replace("_","?")))

            # MappedDpe
            current_row[9] = mappings[mapping_spaces.index(key)]
            
            output_df.loc[len(output_df.index)] = (current_row) 
            
            current_tags = dict(zip(output_columns,current_row))
            ET.SubElement(root, 'OBJ', attrib=current_tags)

In [23]:
# output to xml file
xml_str = minidom.parseString(ET.tostring(root,xml_declaration=None)).toprettyxml(indent="   ")
xml_str = xml_str.split("<?xml version=\"1.0\" ?>")[1][1:]
output_filename = str(asdr_filename.split(".")[0]) + ".xml"
with open(output_filename, "w") as f:
    f.write(xml_str)